<a href="https://colab.research.google.com/github/raviteja1432/NNDL/blob/main/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import pandas as pd
import numpy as np
import math
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
data = pd.read_excel("/content/drive/MyDrive/loaddata.xlsx")
data.head()

,Date,Time,Load (kW)
0,01.09.2018,00:00:00,5551.82208
1,NaN,01:00:00,4983.17184
2,NaN,02:00:00,4888.39680
3,NaN,03:00:00,5072.95872
4,NaN,04:00:00,5196.25980


In [36]:
x = []
y = []
for i in range(24, data.shape[0]):
    x.append(data['Load (kW)'].iloc[i-24])
    y.append(data['Load (kW)'].iloc[i])
data = pd.DataFrame({'Previous Day Load in kw' : x, 'Present Day Load in kw' : y})
data.head()


,Previous Day Load in kw,Present Day Load in kw
0,5551.82208,4931.26380
1,4983.17184,4775.53968
2,4888.39680,4713.81120
3,5072.95872,4689.02628
4,5196.25980,4844.75040


In [37]:
#normalizing
maxX = np.max(data['Previous Day Load in kw'])
minX = np.min(data['Previous Day Load in kw'])
maxY = np.max(data['Present Day Load in kw'])
minY = np.min(data['Present Day Load in kw'])

data['Previous Day Load in kw'] = (data['Previous Day Load in kw'] - minX) / (maxX - minX)
data['Present Day Load in kw'] = (data['Present Day Load in kw']- minY) / (maxY - minY)


In [38]:
from sklearn.model_selection import train_test_split
x = data['Previous Day Load in kw']
y = data['Present Day Load in kw']
trainX, testX, trainY, testY = train_test_split(x, y, test_size=0.1, random_state=101)


In [39]:
#model training
m = np.random.uniform(-5, 5)
c = np.random.uniform(-5, 5)
eta = 0.01
epochs = 200
vm = 0
vc = 0
v = 0.5

# Loop for epochs
for _ in range(epochs):
    
    for i in range(trainX.shape[0]):
        # Calculating Grad M
        gradM = -1 * (trainY.iloc[i] - m * trainX.iloc[i] - c) * trainX.iloc[i]
    
        # Calculating Grad C
        gradC = -1 * (trainY.iloc[i] - m * trainX.iloc[i] - c)
        
        # Calculating updated values of vm and vc
        vm = (v * vm) - (eta * gradM)
        vc = (v * vc) - (eta * gradC)
        
        # Updating m and c values
        m += vm
        c += vc

# Print model parameters after training
print(f'm = {m}\nc = {c}')


m = 0.7171132537325571
c = 0.12511078784145233


In [40]:
train_pred = [m * trainX.iloc[i] + c for i in range(trainX.shape[0])]
test_pred = [m * testX.iloc[i] + c for i in range(testX.shape[0])]

# Denormalization of Output Values
train_pred = [i * (maxY - minY) + minY for i in train_pred]
test_pred = [i * (maxY - minY) + minY for i in test_pred]
trainY = [i * (maxY - minY) + minY for i in trainY]
testY = [i * (maxY - minY) + minY for i in testY]
import math
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error
rmse = math.sqrt(mean_squared_error(train_pred, trainY)) 
print("testing error")
print('Root mean square error', rmse) 
mse = (mean_squared_error(train_pred,trainY)) 
print('Mean square error', mse) 
mae=mean_absolute_error(train_pred,trainY)
print('Mean absolute error', mae)


#Testing Accuracies
rmse = math.sqrt(mean_squared_error(test_pred,testY ))
print("testing error") 
print('Root mean square error', rmse) 
mse = (mean_squared_error(test_pred,testY))
print('Mean square error', mse) 
mae=mean_absolute_error(test_pred,testY)
print('Mean absolute error', mae)

testing error
Root mean square error 720.8559136062257
Mean square error 519633.2481810664
Mean absolute error 533.2583331279448
testing error
Root mean square error 705.0519097029445
Mean square error 497098.19537576893
Mean absolute error 546.2799015481901


In [42]:
#checking the difference between actal value and predicted value of training data
pd.DataFrame({'Actual value' : trainY, 'Predicted value' : train_pred})


,Actual value,Predicted value
0,4221.85392,4606.326987
1,7609.43808,6503.071349
2,5847.05880,5918.219480
3,4672.50300,4994.327911
4,7883.78688,6613.066425
...,...,...
1939,5830.37964,5678.778979
1940,6110.49600,5870.823228
1941,5289.94368,5445.151226
1942,7962.97392,7536.399077


In [43]:
#checking the difference between actal value and predicted value of testing data
pd.DataFrame({'Actual value' : testY, 'Predicted value' : test_pred})


,Actual value,Predicted value
0,6161.624640,5671.401261
1,5761.792440,6900.126746
2,7390.270800,6397.771184
3,4597.368840,5845.783699
4,5242.712040,5959.132283
...,...,...
211,6764.319072,7487.549637
212,7390.270800,6726.415010
213,6550.077600,6545.325555
214,5722.043040,5567.554283


In [45]:
x = float(input('Enter the load of previous day at present hour : '))
x = (x - minX) / (maxX - minX)
prediction = m * x + c
prediction = (prediction * (maxY - minY)) + minY
print('Predicted load at present hour :', prediction)


Enter the load of previous day at present hour : 6550.077
Predicted load at present hour : 6336.2897665212495
